In [31]:
import pandas as pd
import seaborn as sns
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
import hashlib
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn import datasets
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.multiclass import OneVsRestClassifier

In [52]:
df = pd.read_csv('C:/users/hoang/Desktop/Study/CAP4770/Project/USVideos/USVideos.csv'
#                  names = ['video_id', 'trending_date', 'title', 'channel_title','category_id','publish_time',
# 'tags',
# 'views',
# 'likes',
# 'dislikes',
# 'comment_count',
# 'thumbnail_link',
# 'comments_disabled',
# 'ratings_disabled',
# 'video_error_or_removed',
# 'description']
                )

In [56]:
data = df[(df['title'] != '[none]' ) & (df['tags'] != '[none]' ) \
          & (df['views'] != '[none]' )& (df['likes'] != '[none]' ) \
          &(df['dislikes'] != '[none]' ) \
          &(df['comment_count'] != '[none]' )]

In [ ]:
data.info()

In [57]:
preprocessed_data = data.drop(['description', 'thumbnail_link', 'comments_disabled', 'ratings_disabled', 'video_error_or_removed', 'publish_time', 'channel_title', 'trending_date', 'video_id', 'tags', 'title'], axis=1)
preprocessed_data.head()
text_data = data.drop(['views', 'likes', 'dislikes', 'comment_count', 'thumbnail_link', 'comments_disabled', 'ratings_disabled', 'video_error_or_removed','video_id', 'publish_time'], axis=1)

In [58]:
X = preprocessed_data.drop('category_id', axis=1)
y = preprocessed_data['category_id']

In [74]:
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 
   
ps = PorterStemmer() 
def getWords(col:str, data_notNull):
    common_words = set(stopwords.words('english'))
    tagsArray = text_data[col]
    tag2D = []
    tagsArray
    
    for (i,tag) in enumerate(tagsArray):
        lineArray = tag.split('|')
        words = []
        for sentence in lineArray:
            words = words + re.split(r'\W+', sentence.strip())
           
        tag2D.append(words)
        
    for i in range(len(tag2D)):
        temp = [u for u in tag2D[i] if u.lower() not in common_words] 
        temp = " ".join(ps.stem(str(x)) for x in temp)
        tag2D[i] = temp
                
    return tag2D

In [75]:
arr2d_words = getWords('title',text_data)
orr2d_words = getWords('tags', text_data)

In [76]:
X['modified_title'] = arr2d_words
X['modified_tags'] = orr2d_words

In [77]:
le = LabelEncoder()
le.fit(X['modified_title'])
X['labeled_title'] = le.transform(X['modified_title'])
oc = LabelEncoder()
oc.fit(X['modified_tags'])
X['labeled_tags'] = oc.transform(X['modified_tags'])

In [78]:
X = X.drop('modified_title', axis=1)
X = X.drop('modified_tags', axis=1)

In [16]:
import time

In [ ]:
X

In [17]:
def roc_scores(y_test, y_score, n_classes):
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    return fpr, tpr, roc_auc

def histogram_feature(feature):
    fig, ax = plt.subplots()
    plt.hist(data[feature])
    plt.xlabel(feature)
    plt.ylabel('No. of Videos')

def histogram_log_feature(feature):
    fig, ax = plt.subplots()
    plt.hist(data[feature], log=True)
    plt.xlabel(feature)
    plt.ylabel('Log(No. of Videos)')

def bivariate_scatterplot(feature1, feature2):
    fig, ax = plt.subplots()
    plt.scatter(x=data[feature1], y=data[feature2], edgecolor="#000000", linewidths=0.5)
    ax.set(xlabel=feature1, ylabel=feature2)

def kdeplot(feature, data):
    sns.kdeplot(data[feature], shade=true)

def countplot(feature, data):
    sns.countplot(x=feature, data=data, linewidth=5)

#generates an ROC curve for one feature
#args are false positive and true positve rate array for one class
#and the roc_auc for that class
def single_attribute_roc_curve(fpr_arr, tpr_arr, roc_auc_arr):
    
    plt.figure()
    lw = 2
    plt.plot(fpr_arr, tpr_arr, color='darkorange',
             lw=lw, label='ROC curve (area = %0.2f)' % roc_auc_arr)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()
    


def full_attr_roc_curve(fpr, tpr, roc_auc, y_test, y_score, n_classes):
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    #grabs all unique false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

    #interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= n_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    # Plot all ROC curves
    lw = 2
    plt.figure()
    plt.plot(fpr["micro"], tpr["micro"],
             label='micro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["micro"]),
             color='deeppink', linestyle=':', linewidth=4)

    plt.plot(fpr["macro"], tpr["macro"],
             label='macro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["macro"]),
             color='navy', linestyle=':', linewidth=4)

    colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'tan', 'rosybrown', 'lightcoral', 'maroon', 'peru',
                   'darkorange', 'gold', 'darkkhaki', 'lawngreen', 'turquoise', 'lightcyan', 'steelblue',
                    'slategray','indigo', 'violet', 'salmon', 'dodgerblue', 'darkgoldenrod' , 'burlywood', 'lime',
                   'yellow', 'bisque', 'forestgreen', 'aquamarine', 'indianred', 'mediumorchid', 'cadetblue', 'sandybrown',
                   'peachpuff'])
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
                 label='ROC curve of class {0} (area = {1:0.2f})'
                 ''.format(i, roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Multi class ROC curve')
    plt.legend(loc="lower right")
    
    plt.show()

In [79]:
import json

with open("./US_category_id.json") as f:
    categories = json.load(f)['items']
    
categories

category_list = []
category_map = {}
for cat in categories:
    category_id = int(cat['id'])
    category_map[category_id] = cat['snippet']['title']

for cat in categories:
    category_id = int(cat['id'])
    category_list.append(category_id)

# y = label_binarize(y, classes=y.unique().tolist())
# n_classes = y.shape[1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Ox_train, Ox_test, Oy_train, Oy_test = train_test_split(X,y, test_size=0.35)

text_features_train = X_train.drop(['views', 'likes', 'dislikes', 'comment_count'], axis=1)
text_features_test = X_test.drop(['views', 'likes', 'dislikes', 'comment_count'], axis=1)
X_train = X_train.drop(['labeled_title', 'labeled_tags'], axis=1)
X_test = X_test.drop(['labeled_title', 'labeled_tags'], axis=1)

In [80]:
X_train = X_train.reset_index().drop('index', axis=1)
X_test = X_test.reset_index().drop('index', axis=1)

In [81]:
quantile_scaler = QuantileTransformer(output_distribution='uniform').fit(X_train)
quantile_transformer = QuantileTransformer(output_distribution='uniform').fit(X_train)
Xq_train = quantile_scaler.transform(X_train)
Xq_test = quantile_scaler.transform(X_test)

In [82]:
Xq_train = quantile_scaler.transform(X_train)
Xq_test = quantile_scaler.transform(X_test)

In [83]:
payload = text_features_train.reset_index().drop('index',axis=1).join(pd.DataFrame(Xq_train, columns=['views','likes','dislikes','comment_count']))
payload_test = text_features_test.reset_index().drop('index',axis=1).join(pd.DataFrame(Xq_test, columns=['views','likes','dislikes','comment_count']))

In [89]:
from sklearn.ensemble import RandomForestClassifier
start_time = time.time()
model = RandomForestClassifier(n_estimator = 100).fit(payload, y_train)
print(time.time() - start_time)

TypeError: __init__() got an unexpected keyword argument 'n_estimator'

In [85]:
predict = model.predict(payload_test)

In [88]:
print (accuracy_score(y_test, predict))
from sklearn.metrics import recall_score
recall_score(y_test, predict, average='macro')

0.9063808194849676


0.8838253720197999